# Playing with generator

In [2]:
#%%timeit
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np
import sys
import copy
from pathlib import Path
#from src.envs.generators import RandomRecurrence
#from src.envs.encoders import Equation, IntegerSeries, RealSeries
from sympy import *
import pickle
from collections import defaultdict, OrderedDict
import math

%load_ext autoreload
%autoreload 2
#%cd ~/recur/notebooks
sys.path.append('/private/home/sdascoli/recur')
import src
from src.envs.generators import RandomRecurrence
from src.envs.encoders import  Equation, FloatSequences, IntegerSequences
from src.envs.recurrence import RecurrenceEnvironment


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
class params:
    float_precision = 8
    max_len = 30
    min_len=10
    max_degree = 6
    max_number = 1e100
    float_sequences = True
    output_numeric = False
    max_exponent = 100
    init_scale = 10
    prob_const=1/3
    prob_n = 1/3
    max_int = 10
    prob_rand = 0.05
    dimension=1
    int_base=10000
    n_predictions=5
    float_tolerance=0.1
    more_tolerance="0.1"
    max_ops=10
    max_token_len=200
    eval_noise=False
    eval_input_length_modulo = -1
    operators_to_remove=""
    required_operators=""
    extra_unary_operators=""
    extra_binary_operators=""
    extra_constants=""
    float_constants=None
    use_sympy=False
    simplify=False
    mantissa_len = 3
    
generator = RandomRecurrence(params)
env = RecurrenceEnvironment(params)
env.rng = np.random.RandomState(0)
import time 

SPECIAL_WORDS = ["EOS", "PAD", "(", ")", "SPECIAL"]

if params.float_sequences:
    input_encoder = FloatSequences(params)
else:
    input_encoder = IntegerSequences(params)
input_words = SPECIAL_WORDS+sorted(list(set(input_encoder.symbols)))

if params.output_numeric:
    if params.float_sequences:
        output_encoder = FloatSequences(params)
    else:
        output_encoder = IntegerSequences(params)
else:
    output_encoder = Equation(params, generator.symbols)
print(env.output_word2id)
print(env.output_id2word)

deb = time.time()
k_trials=1000
successful_trials=0
size_input=[]
size_output=[]
infos_stats = {}
#n_ops_stats = {i:0 for i in range(1,(params.max_ops+1)*params.dimension)}

for k in range(k_trials):
    
    x, y, tree, infos = env.gen_expr(False, params.eval_input_length_modulo)
    if tree is not None:
        for info in infos:
            if info not in infos_stats:
                infos_stats[info]=np.zeros((1000,))
            for val in infos[info]:
                infos_stats[info][val]+=1

        successful_trials+=1
        size_input.append(len(x[0]))
        size_output.append(len(y[0]))

size_input=np.array(size_input)
size_output=np.array(size_output)
#n_valid_equations = sum(list(n_ops_stats.values()))
for info in infos_stats:
    non_null_points = np.where(infos_stats[info]!=0.)[0]
    values_non_null_points = infos_stats[info][non_null_points]
    count = values_non_null_points.sum()
    infos_stats[info] = np.concatenate([np.expand_dims(non_null_points,-1), np.expand_dims(values_non_null_points/count,-1)],-1)

print("Percentage of successful generation: {}".format(successful_trials/k_trials*100))
print("Time: {}".format(time.time()-deb))
print(infos_stats)
print("Average sizes: input {}, output {}".format(size_input.max(), size_output.max()))


{'-1': 0, '-10': 1, '-2': 2, '-3': 3, '-4': 4, '-5': 5, '-6': 6, '-7': 7, '-8': 8, '-9': 9, '0': 10, '1': 11, '10': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, 'INT+': 21, 'INT-': 22, 'abs': 23, 'add': 24, 'arccos': 25, 'arcsin': 26, 'arctan': 27, 'cos': 28, 'div': 29, 'e': 30, 'euler_gamma': 31, 'exp': 32, 'inv': 33, 'log': 34, 'mul': 35, 'n': 36, 'pi': 37, 'pow': 38, 'rand': 39, 'sin': 40, 'sqr': 41, 'sqrt': 42, 'sub': 43, 'tan': 44, 'x_0_0': 45, 'x_0_1': 46, 'x_0_2': 47, 'x_0_3': 48, 'x_0_4': 49, 'x_0_5': 50, 'x_0_6': 51, '|': 52, 'EOS': 53, 'PAD': 54, '(': 55, ')': 56, 'SPECIAL': 57, 'OOD_unary_op': 58, 'OOD_binary_op': 59, 'OOD_constant': 60, '': 60}
{0: '-1', 1: '-10', 2: '-2', 3: '-3', 4: '-4', 5: '-5', 6: '-6', 7: '-7', 8: '-8', 9: '-9', 10: '0', 11: '1', 12: '10', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: 'INT+', 22: 'INT-', 23: 'abs', 24: 'add', 25: 'arccos', 26: 'arcsin', 27: 'arctan', 28: 'cos', 29: 'div', 30

In [ ]:
python train.py --export_data True --dump_path=/private/home/pakamienny/Research_2/recur_datasets/all_domain/ --cpu True --max_epoch 2 --eval_input_length_modulo 5 --epoch_size 10000 

In [ ]:
x = np.array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 566.
             ])
np.argwhere(x!=0.)

# Checking encoder

In [32]:
x, y, tree, infos = env.gen_expr(False)
print(x,y,tree)

[['-', 'N9221', 'E-3', '-', 'N9586', 'E-3', '-', 'N4554', 'E-3', '-', 'N2530', 'E-3', '-', 'N7061', 'E-3', '+', 'N2104', 'E-3', '+', 'N1727', 'E-3', '+', 'N7105', 'E-3', '+', 'N9656', 'E-3', '+', 'N4364', 'E-3', '+', 'N1168', 'E-2', '+', 'N1099', 'E-2', '+', 'N1768', 'E-2', '+', 'N2035', 'E-2', '+', 'N1468', 'E-2', '+', 'N2242', 'E-2', '+', 'N2172', 'E-2', '+', 'N2851', 'E-2', '+', 'N3120', 'E-2', '+', 'N2547', 'E-2', '+', 'N3329', 'E-2', '+', 'N3258', 'E-2', '+', 'N3941', 'E-2', '+', 'N4210', 'E-2', '+', 'N3635', 'E-2', '+', 'N4420', 'E-2', '+', 'N4349', 'E-2', '+', 'N5033', 'E-2', '+', 'N5303', 'E-2']] [['sub', 'sub', 'add', 'n', 'abs', 'sqr', 'INT-', '3', 'div', 'INT+', '3', 'x_0_5', 'add', 'INT-', '2', 'sub', 'n', 'x_0_5']] (((n add abs((-3)**2)) sub (3 div x_0_5)) sub (-2 add (n sub x_0_5)))


In [33]:
encoder = Equation(params, generator.symbols)
print(tree.infix())
encoded = encoder.encode(tree)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded.infix())

(((n add abs((-3)**2)) sub (3 div x_0_5)) sub (-2 add (n sub x_0_5)))
['sub', 'sub', 'add', 'n', 'abs', 'sqr', 'INT-', '3', 'div', 'INT+', '3', 'x_0_5', 'add', 'INT-', '2', 'sub', 'n', 'x_0_5']
(((n add abs((-3)**2)) sub (3 div x_0_5)) sub (-2 add (n sub x_0_5)))


In [32]:
x = [np.exp(n) for n in range(10)]
encoder = FloatSequences(params)
print(x)
encoded = encoder.encode(x)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded)

[1.0, 2.718281828459045, 7.38905609893065, 20.085536923187668, 54.598150033144236, 148.4131591025766, 403.4287934927351, 1096.6331584284585, 2980.9579870417283, 8103.083927575384]
['+', 'N100', 'N0', 'N0', 'E-8', '+', 'N271', 'N828', 'N183', 'E-8', '+', 'N738', 'N905', 'N610', 'E-8', '+', 'N200', 'N855', 'N369', 'E-7', '+', 'N545', 'N981', 'N500', 'E-7', '+', 'N148', 'N413', 'N159', 'E-6', '+', 'N403', 'N428', 'N793', 'E-6', '+', 'N109', 'N663', 'N316', 'E-5', '+', 'N298', 'N95', 'N799', 'E-5', '+', 'N810', 'N308', 'N393', 'E-5']
[0.0001, 2.71828183, 7.3890561, 20.085536899999997, 54.59815, 148.413159, 403.428793, 1096.63316, 298.95799000000005, 8103.083930000001]
